# 如何处理速率限制

:::info 前提条件

本指南假设你熟悉以下概念：
- [聊天模型](/docs/concepts/chat_models)
- [LLMs](/docs/concepts/text_llms)
:::


你可能会遇到因为发送了太多请求而被模型提供商API限制速率的情况。

例如，当你在测试数据集上运行多个并行查询来对聊天模型进行基准测试时，可能会发生这种情况。

如果你遇到这种情况，你可以使用速率限制器来帮助将请求速率匹配到API允许的速率。

:::info 需要 ``langchain-core >= 0.2.24``

这个功能是在 ``langchain-core == 0.2.24`` 中添加的。请确保你的包是最新的。
:::

## 初始化速率限制器

Langchain内置了一个内存速率限制器。这个速率限制器是线程安全的，可以在同一进程中的多个线程之间共享。

提供的速率限制器只能限制每单位时间的请求数量。如果你还需要基于请求大小进行限制，它将无法帮助你。

In [ ]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.1,  # <-- 超级慢！我们每10秒只能发送一个请求！！
    check_every_n_seconds=0.1,  # 每100毫秒唤醒一次，检查是否允许发送请求
    max_bucket_size=10,  # 控制最大突发大小
)

## 选择一个模型

选择任意模型，并通过`rate_limiter`属性传入速率限制器。

In [2]:
import os
import time
from getpass import getpass

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass()


from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model_name="claude-3-opus-20240229", rate_limiter=rate_limiter)

让我们确认速率限制器是否正常工作。我们每10秒应该只能调用模型一次。

In [3]:
for _ in range(5):
    tic = time.time()
    model.invoke("hello")
    toc = time.time()
    print(toc - tic)

11.599073648452759
10.7502121925354
10.244257926940918
8.83088755607605
11.645203590393066
